In [60]:
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, TimestampType, FloatType, DoubleType
import datetime
import re
import uuid as uuid
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, unix_time_from_uuid1
from pyspark.sql import SparkSession
import pandas as pd
# import org.apache.spark.sql.cassandra
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import uuid as uuid
spark = SparkSession.builder.appName('historical_fire_plots').getOrCreate()
from pyspark.sql.functions import col, sum, count, lit, dayofmonth, hour, dayofyear, month, year, weekofyear, explode, reverse, lit, udf, substring, to_timestamp, collect_list, monotonically_increasing_id, split, trim, size, to_date
from ast import literal_eval
import sys

In [61]:
spark

In [72]:
df = spark.read.option("header", True).csv('ufc_event_raw_dataset.csv')



df = df.withColumn('event_datetime', to_date(df['event_date'], 'MMMM dd, yyyy'))
yr = year(df['event_datetime'])
mon = month(df['event_datetime'])
dayof = dayofmonth(df['event_datetime'])
df = df.withColumn('event_year', yr)
df = df.withColumn('event_month', mon)
df = df.withColumn('event_day_of_month', dayof)


df.select('event_year', 'event_month', 'event_day_of_month').show(n=10)


# df_eyear = df.groupBy('event_year').agg({'event_year': 'count'}).sort('event_year')

# df_eyear = df_eyear.withColumnRenamed('count(event_year)', 'year_count')

# print(df_eyear.columns)
# df_eyear.show(n=30)


df_eyear = df.groupBy('event_month', 'event_year').agg({'event_month': 'count'}).sort('event_year', 'event_month')

df_eyear = df_eyear.withColumnRenamed('count(event_month)', 'month_count')

print(df_eyear.columns)
df_eyear.show(n=300)







# df_count = df.agg({"event_location" : "count"})

# df_count.show()

# df_loc = df.groupBy("event_location").agg({"event_location" : "count"})
# split_event_loc = split(df_loc['event_location'], ',')
# df_loc = df_loc.withColumn('country', trim(split_event_loc[size(split_event_loc)-1]))
# df_loc = df_loc.withColumn('province_or_city', trim(split_event_loc[size(split_event_loc)-2]))
# df_loc = df_loc.withColumn('city', trim(split_event_loc[0]))
# df_loc = df_loc.withColumnRenamed('count(event_location)', 'location_event_count')

# df_loc.show()
# df_loc.printSchema()
# df_loc.agg({'location_event_count' : 'sum'}).show()
# df_loc.show()







+----------+-----------+------------------+
|event_year|event_month|event_day_of_month|
+----------+-----------+------------------+
|      2021|         12|                11|
|      2021|         12|                 4|
|      2021|         11|                20|
|      2021|         11|                13|
|      2021|         11|                 6|
|      2021|         10|                30|
|      2021|         10|                23|
|      2021|         10|                16|
|      2021|         10|                 9|
|      2021|         10|                 2|
+----------+-----------+------------------+
only showing top 10 rows

['event_month', 'event_year', 'month_count']
+-----------+----------+-----------+
|event_month|event_year|month_count|
+-----------+----------+-----------+
|          3|      1994|          1|
|          9|      1994|          1|
|         12|      1994|          1|
|          4|      1995|          1|
|          7|      1995|          1|
|          9|    